In [1]:
# !pip install transformers datasets evaluate torch numpy pandas SentencePiece
# !pip install transformers[torch]
# !pip install accelerate==0.34.2
# !pip show accelerate
# !pip install absl-py rouge_score

  Using cached datasets-3.0.1-py3-none-any.whl.metadata (20 kB)
  Using cached evaluate-0.4.3-py3-none-any.whl.metadata (9.2 kB)
Using cached datasets-3.0.1-py3-none-any.whl (471 kB)
Using cached evaluate-0.4.3-py3-none-any.whl (84 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.0.1
    Uninstalling accelerate-1.0.1:
      Successfully uninstalled accelerate-1.0.1
Name: accelerate
Version: 0.34.2
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: 
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 78.6 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=2c499110a875e649890257cfa65c70e6724081ed77eba9

In [2]:
from transformers import BertTokenizer, EncoderDecoderModel
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, Trainer, EarlyStoppingCallback
from transformers import ProgressCallback
from transformers import TrainerCallback
from datasets import Dataset, DatasetDict, load_dataset
from evaluate import load
import torch
import os
from os import listdir
from os.path import isfile, join
import json
import re
import numpy as np
import pandas as pd
import requests

In [3]:
path = os.getcwd()

In [4]:
# Load dataset from parquet files using load_dataset
dataset = load_dataset('parquet', data_files={'train': 'dataset/used/train.parquet', 
                                               'test': 'dataset/used/test.parquet', 
                                               'dev': 'dataset/used/dev.parquet'})

# Drop unused columns
columns_to_drop = ['id', 'url','clean_article','clean_summary','extractive_summary','ext_clean_article','ext_clean_summary','extractive_summary_sentences','ext_extractive_summary','prep_clean_article_no_sw']

dataset['train'] = dataset['train'].remove_columns(columns_to_drop)
dataset['test'] = dataset['test'].remove_columns(columns_to_drop)
dataset['dev'] = dataset['dev'].remove_columns(columns_to_drop)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating dev split: 0 examples [00:00, ? examples/s]

In [5]:
dataset['train']

Dataset({
    features: ['prep_clean_article', 'prep_clean_summary', 'prep_extractive_summary'],
    num_rows: 193883
})

In [6]:
train_sample = dataset["train"].shuffle(seed=42).select(range(int(5000)))
dev_sample = dataset["dev"].shuffle(seed=42).select(range(int(1000)))
test_sample = dataset["test"].shuffle(seed=42).select(range(int(1000)))

# train_sample = dataset["train"].shuffle(seed=42).select(range(int(len(dataset["train"]) * 0.005)))
# dev_sample = dataset["dev"].shuffle(seed=42).select(range(int(len(dataset["dev"]) * 0.05)))
# test_sample = dataset["test"].shuffle(seed=42).select(range(int(len(dataset["test"]) * 0.05)))

In [7]:
print(f"Train size: {len(train_sample)}")
print(f"Val size: {len(dev_sample)}")
print(f"Test size: {len(test_sample)}")

Train size: 50
Val size: 5
Test size: 5


In [8]:
dataset_dict = DatasetDict({
    'train': train_sample,
    'val': dev_sample,
    'test': test_sample
})

In [9]:
tokenizer = BertTokenizer.from_pretrained("cahya/bert2bert-indonesian-summarization")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token
model = EncoderDecoderModel.from_pretrained("cahya/bert2bert-indonesian-summarization")

tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/230k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.09k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/999M [00:00<?, ?B/s]

EncoderDecoderModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


In [10]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [11]:
prefix = ""
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["prep_clean_article"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    labels = tokenizer(text_target=examples["prep_clean_summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [12]:
tokenized_dataset = dataset_dict.map(preprocess_function, batched=True)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [13]:
from_number = '6285741880658@c.us'

hasil = f'Memulai Pelatihan Model \nTrain size: {len(train_sample)} \nVal size: {len(dev_sample)} \nTest size: {len(test_sample)}'

response = requests.post(
        'https://whatsapp.inspektorat.pekalongankab.go.id/api/sendText/', #rahasia ya ;D
        headers={
            'Content-Type': 'application/json; charset=utf-8',
            'Accept': 'application/json',
            'X-Api-Key': 'KMZWAY87AA' # rahasia
        },
        json={  # Menggunakan parameter `json` untuk mengirim raw JSON
            'chatId': from_number,
            'text': hasil,
            'session': 'NoamChomsky'
        },
        verify=False  # Ini sesuai dengan withoutVerifying() di PHP
    )

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'whatsapp.inspektorat.pekalongankab.go.id'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [14]:
from transformers import TrainerCallback

class CustomCallback(TrainerCallback):
    def on_epoch_end(self, args, state, control, **kwargs):
        # Catat training loss
        training_loss = state.log_history[-1]['loss'] if state.log_history else None
        state.log_history.append({
            'epoch': state.epoch,
            'training_loss': training_loss
        })


In [15]:
for param in model.get_encoder().parameters():
    param.requires_grad = False  # Membekukan semua parameter encoder

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",   
    save_strategy="epoch",         
    logging_dir='./logs',          
    logging_steps=10,              
    learning_rate=3e-5,
    per_device_train_batch_size=4, 
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    weight_decay=0.05,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    load_best_model_at_end=True,
    metric_for_best_model="rouge2",
    greater_is_better=True,
)

In [17]:
rouge = load('rouge')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    # Directly decode predictions
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=False)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=False)

    # Rouge expects newline-separated text
    decoded_preds = ["\n".join(decoded_pred.split()) for decoded_pred in decoded_preds]
    decoded_labels = ["\n".join(decoded_label.split()) for decoded_label in decoded_labels]

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    return {k: round(v, 4) for k, v in result.items()}

In [18]:
torch.set_num_threads(4)  # Mengatur jumlah threads sesuai dengan jumlah core CPU

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["val"],
    compute_metrics=compute_metrics,
    data_collator=data_collator,  # Pastikan ini sudah teratur
    callbacks=[
        CustomCallback(),  
        EarlyStoppingCallback(early_stopping_patience=3),
        ProgressCallback()
    ]
)

In [19]:
trainer.train()

  0%|          | 0/12 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:643: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
0,No log,3.231497,0.476700,0.285800,0.444300,0.476700
1,2.280500,3.045893,0.504300,0.324800,0.477000,0.502300


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 3.231496810913086, 'eval_rouge1': 0.4767, 'eval_rouge2': 0.2858, 'eval_rougeL': 0.4443, 'eval_rougeLsum': 0.4767, 'eval_runtime': 49.1392, 'eval_samples_per_second': 0.102, 'eval_steps_per_second': 0.041, 'epoch': 0.92}


...

  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 3.0458927154541016, 'eval_rouge1': 0.5043, 'eval_rouge2': 0.3248, 'eval_rougeL': 0.477, 'eval_rougeLsum': 0.5023, 'eval_runtime': 46.8268, 'eval_samples_per_second': 0.107, 'eval_steps_per_second': 0.043, 'epoch': 1.85}


There were missing keys in the checkpoint model loaded: ['decoder.cls.predictions.decoder.weight', 'decoder.cls.predictions.decoder.bias'].


{'train_runtime': 226.5006, 'train_samples_per_second': 0.441, 'train_steps_per_second': 0.053, 'train_loss': 2.1642970641454062, 'epoch': 1.85}


TrainOutput(global_step=12, training_loss=2.1642970641454062, metrics={'train_runtime': 226.5006, 'train_samples_per_second': 0.441, 'train_steps_per_second': 0.053, 'total_flos': 32815244642304.0, 'train_loss': 2.1642970641454062, 'epoch': 1.8461538461538463})

In [ ]:
# Menyimpan model
model.save_pretrained("model/bert2bert-cahya")

# Menyimpan tokenizer
tokenizer.save_pretrained("model/bert2bert-cahya")

In [20]:
# Mendapatkan log history dari trainer
log_history = trainer.state.log_history

# Membuat DataFrame dari log_history
df_log_history = pd.DataFrame(log_history)

# Simpan ke file CSV
df_log_history.to_csv('log_history.csv', index=False)

train_runtime_minutes = ''

# Mencari log yang berisi train_runtime
for log in log_history:
    if 'train_runtime' in log:
        train_runtime = log['train_runtime']
        train_runtime_minutes = train_runtime / 60  # Konversi ke menit
        break

In [ ]:
from_number = '6285741880658@c.us'

hasil = f'Selesai Pelatihan Model : {train_runtime_minutes:.2f} menit'

response = requests.post(
        'https://whatsapp.inspektorat.pekalongankab.go.id/api/sendText/', #rahasia ya ;D
        headers={
            'Content-Type': 'application/json; charset=utf-8',
            'Accept': 'application/json',
            'X-Api-Key': 'KMZWAY87AA' # rahasia
        },
        json={  # Menggunakan parameter `json` untuk mengirim raw JSON
            'chatId': from_number,
            'text': hasil,
            'session': 'NoamChomsky'
        },
        verify=False  # Ini sesuai dengan withoutVerifying() di PHP
    )